# Zero-Knowledge Proof of Sudoku Solution

In this example we will demonstrate how ZKPyC can be used to create a zero-knowledge proof of a valid sudoku solution.

In [1]:
from zkpyc import ZKP

In [2]:
zkp = ZKP(modulus="bls12_381", backend="groth16")

In [3]:
from zkpyc.types import Private, Public, Array, field

First, we define the validity function `is_valid` for checking that a solution is valid

In [4]:
N: int = 9 # grid size
M: int = 3 # sub-grid size

def is_valid(grid: Array[Array[int, 9], 9]) -> bool:
    # Check that cell value is in correct range
    for i in range(N):
        for j in range(N):
            assert 1 <= grid[i][j] <= 9, "Each cell must contain a number between 1 and 9."

    # Check that rows have unique values
    for i in range(N):
        for j in range(N):
            for k in range(j + 1, N):
                assert grid[i][j] != grid[i][k], "Each row must contain unique values."

    # Check that columns have unique values
    for j in range(N):
        for i in range(N):
            for k in range(i + 1, N):
                assert grid[i][j] != grid[k][j], "Each column must contain unique values."

    # Check that each 3x3 sub-grid has unique values
    for x in range(N // M):
        for y in range(N // M):
            subgrid: Array[int, 9] = [0 for _ in range(9)]
            idx: int = 0
            for i in range(M):
                for j in range(M):
                    subgrid[idx] = grid[x * M + i][y * M + j]
                    idx += 1
            for i in range(M):
                for j in range(i + 1, M):
                    assert subgrid[i] != subgrid[j], "Each 3x3 sub-grid must contain unique values."
    
    return True

Next, we define the proof of solution function `check_puzzle`

In [5]:
def check_puzzle(witness: Private[Array[int, 81]], sudoku_puzzle: Public[Array[Array[int, 9], 9]]) -> bool:
    grid: Array[Array[int, 9], 9] = [[0 for _ in range(N)] for _ in range(N)]
    for i in range(N):
        for j in range(N):
            grid[i][j] = witness[i * N + j]
    
    # Check that the provided solution matches the original puzzle
    for i in range(N):
        for j in range(N):
            assert grid[i][j] == sudoku_puzzle[i][j] or sudoku_puzzle[i][j] == 0, "Provided solution does not match the original puzzle."

    return True if is_valid(grid) == True else False

## Compile the Zero-Knowledge Proof

After defining the zero-knowledge statement as the function above, we can compile it as follows

In [6]:
zkp.compile(check_puzzle, includes=[is_valid, M, N], global_vars=globals())

8838

In [7]:
crs = zkp.generate_crs(check_puzzle)
zkp.store_crs(check_puzzle, crs) # optional if running locally

Since the `check_puzzle` function compiled successfully (at around 8.8k constraints), we can use it to provide a zero-knowledge proof of any valid solution to a given puzzle

## Prove and Verify a Witness

Let's consider a hard puzzle with lots of blank cells

In [8]:
puzzle: Array[Array[int, 9], 9] = [
    [0, 2, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 6, 0, 0, 0, 0, 3],
    [0, 7, 4, 0, 8, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 3, 0, 0, 2],
    [0, 8, 0, 0, 4, 0, 0, 1, 0],
    [6, 0, 0, 5, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 7, 8, 0],
    [5, 0, 0, 0, 0, 9, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 4, 0],
]

This would take an average person a long time to solve, but luckily we know a solution - and we can prove it too!

### Case 1: Valid Solution

Let's consider the following solution to the puzzle

In [9]:
solution: Private[Array[int, 81]] = [
    1, 2, 6, 4, 3, 7, 9, 5, 8,
    8, 9, 5, 6, 2, 1, 4, 7, 3,
    3, 7, 4, 9, 8, 5, 1, 2, 6,
    4, 5, 7, 1, 9, 3, 8, 6, 2,
    9, 8, 3, 2, 4, 6, 5, 1, 7,
    6, 1, 2, 5, 7, 8, 3, 9, 4,
    2, 6, 9, 3, 1, 4, 7, 8, 5,
    5, 4, 8, 7, 6, 9, 2, 3, 1,
    7, 3, 1, 8, 5, 2, 6, 4, 9,
]

We can first check that this is indeed valid by running the solution and puzzle through the `check_puzzle` function

In [10]:
check_puzzle(solution, puzzle)

True

Once we generate the zero-knowledge proof and send it out, anyone will be able to verify that the solution to the puzzle is correct.

In [11]:
proof = zkp.prove(check_puzzle, solution, puzzle)
zkp.store_proof(check_puzzle, proof) # optional if running locally

In [12]:
zkp.verify(check_puzzle, None, puzzle, return_value=True)

True

And just like that, we did not require the solution to verify its validity

### Case 2: Incompatible Solution

Showing just a valid solution is not a convincing argument that our `check_puzzle` function operates correctly. Let us consider the case where the solution does not match the puzzle by switching the last two rows

In [13]:
incompatible_solution: Private[Array[int, 81]] = [
    1, 2, 6, 4, 3, 7, 9, 5, 8,
    8, 9, 5, 6, 2, 1, 4, 7, 3,
    3, 7, 4, 9, 8, 5, 1, 2, 6,
    4, 5, 7, 1, 9, 3, 8, 6, 2,
    9, 8, 3, 2, 4, 6, 5, 1, 7,
    6, 1, 2, 5, 7, 8, 3, 9, 4,
    2, 6, 9, 3, 1, 4, 7, 8, 5,
    7, 3, 1, 8, 5, 2, 6, 4, 9,
    5, 4, 8, 7, 6, 9, 2, 3, 1,
]

We can check this and see that there is an assertion error

In [14]:
check_puzzle(incompatible_solution, puzzle)

AssertionError: Provided solution does not match the original puzzle.

And just like before, we can provide a zero-knowledge proof of this fact

In [15]:
proof = zkp.prove(check_puzzle, incompatible_solution, puzzle)

In [16]:
zkp.verify(check_puzzle, None, puzzle, return_value=True)

False

### Case 3: Invalid Solutions

Lastly, let's also consider cases where the solution matches the puzzle but is considered invalid.

#### Cell value out of bounds

Let us consider picking a random cell and setting it to 10

In [17]:
invalid_solution_1: Private[Array[int, 81]] = [
    1, 2, 6, 4, 3, 7, 9, 5, 8,
    8, 9, 5, 6, 2, 1, 4, 7, 3,
    3, 7, 4, 9, 8, 5, 1, 2, 6,
    4, 5, 7, 1, 9, 3, 8, 6, 2,
    9, 8, 3, 2, 4, 6, 5, 1, 7,
    6, 1, 2, 5, 7, 8, 3, 10, 4, # 10 is out of bounds
    2, 6, 9, 3, 1, 4, 7, 8, 5,
    5, 4, 8, 7, 6, 9, 2, 3, 1,
    7, 3, 1, 8, 5, 2, 6, 4, 9,
]

In [18]:
check_puzzle(invalid_solution_1, puzzle)

AssertionError: Each cell must contain a number between 1 and 9.

The function outputs correctly and its corresponding zero-knowledge proof can be generated and verified as follows

In [19]:
proof = zkp.prove(check_puzzle, invalid_solution_1, puzzle)

In [20]:
zkp.verify(check_puzzle, None, puzzle, return_value=True)

False

#### Duplicate values

Next, we modify the valid solution by mutating `solution[1][5]` to store the duplicate value 6

In [21]:
invalid_solution_2: Private[Array[int, 81]] = [
    1, 2, 6, 4, 3, 7, 9, 5, 8,
    8, 9, 5, 6, 2, 6, 4, 7, 3,  # Duplicate 6 in second row
    3, 7, 4, 9, 8, 5, 1, 2, 6,
    4, 5, 7, 1, 9, 3, 8, 6, 2,
    9, 8, 3, 2, 4, 6, 5, 1, 7,
    6, 1, 2, 5, 7, 8, 3, 9, 4,
    2, 6, 9, 3, 1, 4, 7, 8, 5,
    5, 4, 8, 7, 6, 9, 2, 3, 1,
    7, 3, 1, 8, 5, 2, 6, 4, 9,
]

In [22]:
check_puzzle(invalid_solution_2, puzzle)

AssertionError: Each row must contain unique values.

Which has the following proof

In [23]:
proof = zkp.prove(check_puzzle, invalid_solution_2, puzzle)

In [24]:
zkp.verify(check_puzzle, None, puzzle, return_value=True)

False

## Finish

And as always, we clear the ZKPyC cache once we are done generating proofs

In [25]:
zkp.cleanup()